In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import re

In [2]:
df = pd.read_csv('data/population_cleaned.csv')

In [3]:
df.head()

,age,nationality,sex,commune,year,value
0,0,S,M,Aeugst am Albis,2010,8
1,1,S,M,Aeugst am Albis,2010,9
2,2,S,M,Aeugst am Albis,2010,7
3,3,S,M,Aeugst am Albis,2010,13
4,4,S,M,Aeugst am Albis,2010,5


In [4]:
#group the ages by categories :{[0, 18), [18, 40), [40, 65), [65, 100)}
d = pd.DataFrame(columns=['commune', 'sex', 'year', 'nationality', 'age', 'value'])
for l, u in [(0, 18), (18, 40), (40, 65), (65, 100)]:
    tmp = df[df['age'] >= l]
    tmp = tmp[tmp['age'] < u]
    tmp = tmp.groupby(['commune', 'sex', 'year', 'nationality'], as_index=False).sum()
    tmp['age'] = int(u)
    d = pd.concat([d, tmp])
for s in ['year', 'age', 'value']:
    d[s] = d[s].astype(int)
df = d

In [5]:
df.head()

,commune,sex,year,nationality,age,value
0,Aadorf,F,2010,A,18,122
1,Aadorf,F,2010,S,18,650
2,Aadorf,F,2011,A,18,120
3,Aadorf,F,2011,S,18,656
4,Aadorf,F,2012,A,18,108


In [6]:
ids =  pd.read_csv('data/data_to_be_processed/ids')
ids.rename(columns={'name':'commune'}, inplace=True)
ids = ids.sort_values('commune', ascending=True)

In [7]:
for year in range (2013, 2016):
    #filter out by year
    tmp = df[df['year'] == year]
    tmp = tmp.drop('year', axis=1)
    #compute total population by municipality
    total = tmp.groupby('commune').sum()
    total = total.drop('age', axis=1)
    total.columns = ['total_inhabitants']
    #for each age category compute the percentage of inhabitants in it and add the row to total
    for i in [18, 40, 65, 100]:
        age = tmp[tmp['age'] == i].groupby('commune').sum()
        age = age.drop('age', axis=1)
        age.columns = ['percentage_%d' %i]
        total = pd.merge(total, age, left_index=True, right_index=True)
        total['percentage_%d' %i] = total['percentage_%d' %i] / total['total_inhabitants']
    #compute percentage of men in each municipality
    men = tmp[tmp.sex == 'M'].groupby('commune').sum()
    men = men.drop('age', axis=1)
    men.columns = ['percentage_men']
    total = pd.merge(total, men, left_index=True, right_index=True)
    total['percentage_men'] =  total['percentage_men'] / total['total_inhabitants']
    #compute percentage of swissin each municipality
    swiss = tmp[tmp.nationality == 'S'].groupby('commune').sum()
    swiss = swiss.drop('age', axis=1)
    swiss.columns = ['percentage_swiss']
    total = pd.merge(total, swiss, left_index=True, right_index=True)
    total['percentage_swiss'] = total['percentage_swiss'] / total['total_inhabitants']
    
    total = total.reset_index(level=0)
    #merge dataframe with municiplity ids
    total = pd.merge(ids, total, left_on='commune', right_on='commune')
    total = total.sort_values('id')
    total['id'] = total['id'].astype(int)
    #round percentages to 2 decimals
    total = total.round(2)
    total.to_csv('data/%d/data_commune.csv' % year, index=False)

In [8]:
total.head()

,id,commune,total_inhabitants,percentage_18,percentage_40,percentage_65,percentage_100,percentage_men,percentage_swiss
20,1,Aeugst am Albis,1981,0.19,0.23,0.42,0.16,0.50,0.87
22,2,Affoltern am Albis,11707,0.18,0.30,0.35,0.17,0.50,0.73
233,3,Bonstetten,5326,0.22,0.26,0.37,0.15,0.49,0.86
849,4,Hausen am Albis,3477,0.20,0.23,0.40,0.17,0.49,0.86
856,5,Hedingen,3659,0.21,0.25,0.39,0.15,0.50,0.85
